In [1]:
import numpy as np
import pandas as pd
import osmnx as ox
import re

In [2]:
schools = pd.read_csv("csv/schools_complete.csv")

In [17]:
schools["postcode"] = schools["postcode"].map(lambda x: " "
                                              .join([i for i in x.split(" ") if i != ""])
                                              .replace("OJB", "0JB")
                                              .replace("OLA", "0LA")
                                              .replace("BT74 4FY", "BT74 4FW")
                                             )

In [15]:
def get_place_details(postcode):
    # Before the following, add params["addressdetails"] = 1 to osmnx.downloader._osm_place_download
    result = ox.downloader._osm_place_download(postcode)
    
    if len(result)==0: print(postcode)
    details = result[0]
    
    try:
        _ = details["address"]["county"]
    except KeyError:
        details["address"]["county"] = "None"
        
    return (details["lat"], details["lon"], details["address"]["county"])

In [18]:
places = schools["postcode"].map(get_place_details)

In [21]:
schools[["lat", "lon", "county"]] = pd.DataFrame(places.tolist(), index=schools.index)

In [23]:
schools.to_csv("csv/schools_complete_clean.csv", index=False)